In [2]:
import numpy as np 
import pandas as pd
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
import pickle as cPickle
from os.path import exists
from pathlib import Path
import csv
import datetime
import pytz
import plotly.express as px
import os

from pathlib import Path 
import time

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
test = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/testing_data_all_20220523.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Columns: 299 entries, Unnamed: 0 to wb2sum
dtypes: float64(290), int64(4), object(5)
memory usage: 1.1+ MB


In [4]:
usgs_reg = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/USGS_region_lookup.csv')
usgs_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NA_L3CODE   11 non-null     object
 1   USGS_Group  11 non-null     object
dtypes: object(2)
memory usage: 304.0+ bytes


In [5]:
region_dict = dict(zip(usgs_reg.NA_L3CODE, usgs_reg.USGS_Group))
test['NewEco3'] = test['NewEco3'].replace(region_dict)
print(test['NewEco3'].unique())

['Xeric' 'Interior Mountains' 'North Coastal Mountains']


In [6]:
test = test.loc[(test['NewEco3'] == 'Xeric')] 
test

,Unnamed: 0,year,month,flow_cfs,comid,statistic,variable,ECO3,NHDPLUSREG,DRAIN_SQKM,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
0,0,2021,11,0.000000,22595619,mean,observed,8.0,18.0,57.7287,...,0.131168,2.213277,1.077336,1.245494,62.235627,42.775919,31.027255,3.117207,2.139399,1.551621
1,1,2021,11,0.351000,22591931,mean,observed,8.0,18.0,10.8657,...,1.677609,5.268198,1.707801,1.453772,77.353606,45.856562,44.561237,4.341184,2.345440,2.250610
2,2,2021,11,0.000000,22590267,mean,observed,8.0,18.0,93.4965,...,2.247079,6.138794,1.721187,1.285342,48.954725,29.751628,24.570276,4.766595,1.745232,1.338936
3,3,2021,11,0.000000,22593497,mean,observed,8.0,18.0,44.1009,...,9.131742,20.093948,6.623057,1.712484,83.987403,47.913295,46.620456,10.463754,3.091707,2.629327
4,4,2021,11,0.000000,22593537,mean,observed,8.0,18.0,244.3446,...,0.649975,3.049139,1.616791,0.946181,57.895884,33.346512,29.546124,4.074301,1.798382,1.533473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,49,2022,4,0.361667,348419,mean,observed,6.0,18.0,37.5984,...,1.136439,2.272878,4.545755,9.237394,410.575778,23.842762,21.886998,138.092517,55.955772,24.564478
464,51,2022,4,440.700000,2495212,mean,observed,6.0,18.0,1094.9470,...,3.921204,7.828248,15.562201,32.083588,664.143180,63.917853,19.560333,368.156697,122.233965,46.003030
466,53,2022,4,15.582000,948020963,mean,observed,6.0,17.0,96.6258,...,2.338589,4.677179,9.376106,19.038726,680.563943,62.397547,41.360802,401.322045,116.703810,50.746868
467,54,2022,4,33.536333,8005975,mean,observed,6.0,18.0,155.5119,...,3.046977,6.210674,12.324649,24.703854,558.068358,46.459739,31.399003,303.683755,94.137890,40.815127


In [7]:
#right now we have Nov 2021 - Apr 2022 data
test['month'].unique()

array([11, 12,  1,  2,  3,  4])

In [8]:
#no null columns
test = test.dropna(axis=0,how='any')
test.isnull().sum().sum()

0

In [9]:
test['flow_cfs'] = test['flow_cfs'] / test['DRAIN_SQKM']

In [10]:
#dropping inessential columns
col = ['comid','Unnamed: 0','year','statistic','variable','NHDPLUSREG','comidyear','ECO3','AggEcoRegion','NewEco3'] 
df = test.drop(columns = col)

#no need label encode as only have one region xeric
#label encoding
# cols_encoding = ['NewEco3']
# labelencoder = LabelEncoder()
# df['NewEco3_label'] = labelencoder.fit_transform(df['NewEco3'])
# df = df.drop(columns=cols_encoding)
# df.reset_index(drop=True, inplace=True)

In [11]:
def x_test(month):  
  test1 = df[df['month']==month]
  x_test = test1.drop(columns = ['flow_cfs']).copy()
  return x_test

def y_test(month):
  test1 = df[df['month']==month]
  y_test = test1.flow_cfs.values
  return y_test

November 2021

In [12]:
pickled_model_usgs_11 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Xeric-all-comids)/USGS_model_reduced_11_r2_0.9315_2022-09-19 09:17:50.760727-07:00.pkl', 'rb'))
print(pickled_model_usgs_11)

RandomForestRegressor(max_depth=5, max_features=17, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [13]:
rf_var = pickled_model_usgs_11.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_11
predictions = rf.predict(x_test(11)) 
r2_test = r2_score(y_test(11)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: 0.79


December 2021

In [14]:
pickled_model_usgs_12 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Xeric-all-comids)/USGS_model_reduced_12_r2_0.9175_2022-09-19 09:24:33.350316-07:00.pkl', 'rb'))
print(pickled_model_usgs_12)

RandomForestRegressor(max_depth=5, max_features=17, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [15]:
rf_var = pickled_model_usgs_12.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_12
predictions = rf.predict(x_test(12)) 
r2_test = r2_score(y_test(12)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s


Reduced r-squared scores: 0.64


[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


January 2022

In [16]:
pickled_model_usgs_1 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Xeric-all-comids)/USGS_model_reduced_1_r2_0.9127_2022-09-19 09:25:40.497484-07:00.pkl', 'rb'))
print(pickled_model_usgs_1)

RandomForestRegressor(max_depth=5, max_features=17, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [17]:
rf_var = pickled_model_usgs_1.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_1
predictions = rf.predict(x_test(1)) 
r2_test = r2_score(y_test(1)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: 0.67


February 2022

In [18]:
pickled_model_usgs_2 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Xeric-all-comids)/USGS_model_reduced_2_r2_0.9108_2022-09-19 09:26:48.527045-07:00.pkl', 'rb'))
print(pickled_model_usgs_2)

RandomForestRegressor(max_depth=5, max_features=17, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [19]:
rf_var = pickled_model_usgs_2.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_2
predictions = rf.predict(x_test(2)) 
r2_test = r2_score(y_test(2)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s


Reduced r-squared scores: -0.36


[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


March 2022

In [20]:
pickled_model_usgs_3 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Xeric-all-comids)/USGS_model_reduced_3_r2_0.8947_2022-09-19 09:27:58.008258-07:00.pkl', 'rb'))
print(pickled_model_usgs_3)

RandomForestRegressor(max_depth=5, max_features=17, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [21]:
rf_var = pickled_model_usgs_3.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_3
predictions = rf.predict(x_test(3)) 
r2_test = r2_score(y_test(3)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: 0.45


April 2022

In [22]:
pickled_model_usgs_4 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Xeric-all-comids)/USGS_model_reduced_4_r2_0.8855_2022-09-19 09:29:08.487035-07:00.pkl', 'rb'))
print(pickled_model_usgs_4)

RandomForestRegressor(max_depth=5, max_features=17, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [23]:
rf_var = pickled_model_usgs_4.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_4
predictions = rf.predict(x_test(4)) 
r2_test = r2_score(y_test(4)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: 0.46


May 2022